# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
file_to_load = "../output_data/cities.csv"
cities_data = pd.read_csv(file_to_load)

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)
 
    

In [148]:
# Plot Heatmap
fig = gmaps.figure()

coordinates = cities_data[["Lat", "Lng"]]
weight = cities_data["Humidity"]

# Create heat layer
heat_layer = gmaps.heatmap_layer(coordinates, weights=weight, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure

fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [11]:
ideal_data = cities_data.loc[(cities_data['Country'] == "AU") & 
                             (cities_data['Max Temp'] < 90 ) & 
                             (cities_data['Max Temp']> 70) &
                             (cities_data['Wind Speed'] < 15) &                             
                             (cities_data['Humidity'] < 70)]
ideal_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
28,Codrington,-38.27,141.97,75.06,62,0,10.67,AU,1604287062
35,New Norfolk,-42.78,147.06,77.00,23,89,3.00,AU,1604287126
66,Mount Gambier,-37.83,140.77,80.60,32,47,14.99,AU,1604287364
74,Hervey Bay,-25.30,152.85,80.01,61,5,10.00,AU,1604287126
116,Hobart,-42.88,147.33,77.00,44,0,6.93,AU,1604287236
138,Broken Hill,-31.95,141.43,82.40,24,22,9.17,AU,1604287232
168,Victoria Point,-27.58,153.30,79.00,53,0,13.87,AU,1604287397
270,Port Lincoln,-34.73,135.87,77.79,38,71,13.13,AU,1604287362
381,Lakes Entrance,-37.88,147.98,71.01,67,47,5.99,AU,1604287248
383,Kiama,-34.68,150.87,73.00,66,85,5.01,AU,1604287188


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [12]:
hotel_df = ideal_data.copy()

#creating a new dataframe that meets my search criteria

In [13]:
hotel_df["Hotel Name"] = ""
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
28,Codrington,-38.27,141.97,75.06,62,0,10.67,AU,1604287062,
35,New Norfolk,-42.78,147.06,77.00,23,89,3.00,AU,1604287126,
66,Mount Gambier,-37.83,140.77,80.60,32,47,14.99,AU,1604287364,
74,Hervey Bay,-25.30,152.85,80.01,61,5,10.00,AU,1604287126,
116,Hobart,-42.88,147.33,77.00,44,0,6.93,AU,1604287236,
138,Broken Hill,-31.95,141.43,82.40,24,22,9.17,AU,1604287232,
168,Victoria Point,-27.58,153.30,79.00,53,0,13.87,AU,1604287397,
270,Port Lincoln,-34.73,135.87,77.79,38,71,13.13,AU,1604287362,
381,Lakes Entrance,-37.88,147.98,71.01,67,47,5.99,AU,1604287248,
383,Kiama,-34.68,150.87,73.00,66,85,5.01,AU,1604287188,


In [137]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

#create a variable/dictionary name 
params = {    
    "radius": 5000,
    "type": "lodging",  #google maps API states "lodging"
    "key": g_key
    }


# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

# get the hotel location from hotel_df and add the lat/lng to the params dictionary
    lat = row['Lat']
    lng = row['Lng']
    location = f"{lat},{lng}"
    params['location'] = location


# assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()    
    print(response)
     
#     extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']

        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 28: Codrington.
{'html_attributions': [], 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': -38.2846732, 'lng': 142.0097987}, 'viewport': {'northeast': {'lat': -38.2834211697085, 'lng': 142.0113643302915}, 'southwest': {'lat': -38.2861191302915, 'lng': 142.0086663697085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'name': 'Codrington Gardens Bed And Breakfast', 'opening_hours': {'open_now': False}, 'photos': [{'height': 344, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/104484835300217390749">Codrington Gardens Bed And Breakfast</a>'], 'photo_reference': 'CmRZAAAAjGfIIlstIYRri9CSYtXk21GlUo4KSvm9APxsfVVN5zaqfJSICGCVwFbluuW_DScd7vKfpKYS6oDSzZ-F_EmL94_dy9DZrrR48TKuut1Uu-fONXKRyeJmC_tO0paQodsWEhDfNJDbENSiX79rw5yoLNAhGhQl2Jv_MFH9okBKDkXPehGCNfcC-g', 'width': 258}], 'place_id': 'ChIJXbYfRg9_naoRibu5kQ0H7O0', 'plus_code': {'compound_code': 'P285+4W Yambuk VIC, Austral

{'html_attributions': [], 'next_page_token': 'AcnaJv0m92_mGWGRQafL9K8FakBeIlRg9zmxlUAWToixRNkS_T7hUpXsV6NpTdCPeEusIDufLc8UkvXjFggcUgbsUG4-DyM4JcIBq_t2XeRjYbgyqmznwz6UTE1jNMw1zka3JRAMxp0I2bNKqHu9b_52qi6NJjzzWz43FAAuSu90ORtoQM_S5VfNbimBm3oG3pjrhUt61xnyh-RmOWXXQYYxvTvPlyLwdixboieO4WUHlB54JZ8S0YH2QgTdjPBUpS-nAWp7lcArvdWA97G2Y4kHDKGVJMQXzfDyxnhziCxwpUlNmjsh6VAKp9V5aqhgnEBPPVuzuXYCg0qEgqkeQvFuQwlI_bVAHurntZa9AgqkUx0VKxCz5tVNb453YicY74TIL8DdajdiF94_3z-4RA8AzqdI9MZ86RCEVt5HlrnV5cpz4mJ2FSQ', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': -37.823155, 'lng': 140.784066}, 'viewport': {'northeast': {'lat': -37.8217466697085, 'lng': 140.7854485802915}, 'southwest': {'lat': -37.8244446302915, 'lng': 140.7827506197085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'name': 'Commodore on the Park', 'opening_hours': {'open_now': True}, 'photos': [{'height': 567, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib

{'html_attributions': [], 'next_page_token': 'AcnaJv0XbN40BRtSWa_aNUq5gQd1ZRXE-CWTL7i11yXN-awhEzmSU2WXLTpYSIusVZqsziArrVe_Sq1vcx0Y9K_TIsDJcwJkDxg0qgqRKemk98jlNDeCFvpU13oBv664mS5skxBz06P_q3YhHnEg630Y2xKsIL88oLOfKbYoslKBU3jcRnNuOhS_qFHQFth3V9GdJbxkXyuHrNKUsy5heMuvATmZdPfYH5FsC9eprWNbKFlcWMLwbMDBT8PpmwQhfzZVYL7RJOIcaK55dTwq3Z3TliRWPVN8ELjidfrQOIGVVmsz2ZDzMMu0Y-HOMFvvs9dlRGmGhGR0UAf5gEIOOdtpQNE81U5tSUmdMDvN-3P7DGGA7G___LZd-9A4LPCEZUTP4d64fkcNWUt1lQqW7WPIV65Mj0Wo856uwFdRyDFMdW4WsA2RMIQ', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': -25.2829399, 'lng': 152.8798937}, 'viewport': {'northeast': {'lat': -25.2814785697085, 'lng': 152.8812121302915}, 'southwest': {'lat': -25.2841765302915, 'lng': 152.8785141697085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'name': 'Shelly Bay Resort', 'opening_hours': {'open_now': False}, 'photos': [{'height': 1358, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib

{'html_attributions': [], 'next_page_token': 'AcnaJv2WvE8BQfODHfaJ5QttzL5yxayqiQUP7L7EI8bqRed9oWEZWvkWiqRpGEk65IJ8EMeanWXjhW6Kqpy1y8a4DF960Rloa5BULUn8DpIa1DRcoadejt8X-KdJ-zQ_J5sdriLd44WbTaV0GHkyqCZdjtkEYJatVQwW29cSdV-yxRkBn5160_5nPymt10VzaPN_OzhYZepCbeGPc9sbasAmNXMNi-AKZE703fjOu_tDLWMQsumxLA2DGpTXc1tbB_nwvCNWrE4RiejN1GhllU493AoYYVSijzvQaLyqf5qyTWK_JNjraBYqHtaXi-rkfS6C4ob_MLAlpUeWvnmbF7UZw1YC7o_GUwgNrFgX4l6n_K4tIt1xYoFbJvCxqdEkEI949ketp4A-WuWBsY_dZJjIkNUsaqX4wt_RS10n0xgEPOwaWxtlt9U', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': -42.89114989999999, 'lng': 147.3296669}, 'viewport': {'northeast': {'lat': -42.8897508697085, 'lng': 147.3310552802915}, 'southwest': {'lat': -42.8924488302915, 'lng': 147.3283573197085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'name': 'St Ives Hobart Accommodation Tasmania', 'photos': [{'height': 2428, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/1133546351

{'html_attributions': [], 'next_page_token': 'AcnaJv0tBgtw3fO11rhGzhT1NfNXHsjMeK7K_Xp7507qKInIlN_2Pl-PrCLx202rF07_NOgo_KT_ZkJPNg8zWRBdG-8hHT4LLjQjTdL2lF1Swn9_qssSaE8NifWbVZw9zuHCnQkivE-MegJHArAUzXpwtd7kwoHU2gtwNUSCS5EU6WqaKci9JxOr4EHTp9MYtJ-fmFATE0s9kGuBLfNmSEUug23Z3SKLDya_APXOCWsVPW435JnxNh71SlT7Jm0OBNKVp3Mk12vzDvs_e9r8zS0omXZNYs22s_AK8ygEmduksBMaEtTcAEiMPd-Z038XfKNc-M9Bes2ctha6GFIVUT9ePg9uFjYYnzxyn2lpSaTycFvrk331Hgfwbgnzml5Hz8vhyagWXpFGixpJQRCE5eZiv8R5vmh1Ki-HFnOBUUUYZgUgIR5u5PU', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': -31.9622512, 'lng': 141.4598159}, 'viewport': {'northeast': {'lat': -31.9607553197085, 'lng': 141.4610380302915}, 'southwest': {'lat': -31.9634532802915, 'lng': 141.4583400697085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'name': 'Daydream Motel & Cottages', 'opening_hours': {'open_now': False}, 'photos': [{'height': 2988, 'html_attributions': ['<a href="https://maps.google.com/maps

...

{'html_attributions': [], 'next_page_token': 'AcnaJv01DxjAMDFxEhodGhDM6r7yg61bRxefXxgr0aVDgUeIn5ht38XuknfbFlXj8zu2NIpKLa6R6IcVx7fl7KfvjSmUvD92AdL_uV8S0XxnQVJmA2bXx30fH9kF1HLgCMNKF0KZsk_mbUFrNF4zJmBQ6Wy-E4zOH9dA9W20xYInJlRDALmgAHVjIfV2-Od7-VJlJ2HvTOgCuEJ2kdorZ0Spq8e1sVbEeSjEfAAidcmrBOZTJV0Wpqs9hC-l_aCb4NBsgoYGsUdFmETY7YNKT1Jf_GCAkPqxLoG8F3gEMPqu1VQeDhvHkMkLCjdLSnFbM5CAD1KgcVez8K6t1N-0DT6KXFye2uL8UqOCFjxBdyFM7aFn-yXrUddvWP2smxaBcv-GAZ1HlPmtxR2WtmhLPicbuMI9W1nhA0MzRW7nF2PBI3-z-PKP7tI', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': -34.7178306, 'lng': 135.8544911}, 'viewport': {'northeast': {'lat': -34.71651516970849, 'lng': 135.8555577802915}, 'southwest': {'lat': -34.71921313029149, 'lng': 135.8528598197085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'name': 'Port Lincoln Hotel', 'photos': [{'height': 2000, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/100236087803320022517">Port Lincol

{'html_attributions': [], 'next_page_token': 'AcnaJv1Jtq69YDPi2PQLpdExaD2ny-wteKLRQSyFxv_f75qhT8VxcjRKBbKo3HGSYYPBQ2cOqLi2rjJEqlovUgvBFZLYhbHveXNWqouKZd-FxhSCcx6RRZhT_sIe0jp3MLHlOjRy9DB5NQslT8-IuwPne3oAwE0fJ0sDNbZXQAtH5quOfkwBp3_Xi4eIgYKysI8xjdaSvYT_xaWKRMnAdhLRIVWPP_zvBsK8ErPRY9RlGnH-ETnZgYvL-SU2CnpAbnqaEvUoj5Xt7uc1QdUzRlx_goN9I7fg--UtUzuWsGmxm1n67QWfkIpp7LeRE1NR9R0c6mAXXNa3A6BQ9Hp6hs8tOrl_XyIMbSU0XbLLU5J3E9I1fKDazAJqY-OIQKA_rTb2bxpq6jNdlYxWw7TDRTh0MTkCC__G0ZaZSIpI_5t3aiu9lVXJbdo', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': -37.8826838, 'lng': 147.9746235}, 'viewport': {'northeast': {'lat': -37.88145526970849, 'lng': 147.9760243802915}, 'southwest': {'lat': -37.88415323029149, 'lng': 147.9733264197085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'name': 'The Esplanade Resort & Spa', 'opening_hours': {'open_now': True}, 'photos': [{'height': 667, 'html_attributions': ['<a href="https://maps.google.com/map

{'html_attributions': [], 'next_page_token': 'AcnaJv3mQARQ54LU_Aug2BG0dAQhMrmUl6Ref66PhRZ3UHpJ-2Z2c4449g4n18vJRQ2xW2LJp1rHHqTTKOiEXuV3PmsLXB5I_DD1ysswDHzOv80Mkb3mIeJv90vfciamMJE0f_iJ0dyfeIct9wSs5pJHaaRPg1o9qskApQdbC_aeIdLQ4fn78iJoGnJVSIlaJWh014URMzK34Cj_2c0SxT8WMb4sOrLhz0NTBBsIWvZSLbVDyMh8VtduSKHlyhWqfDXmy08OJyeWfeQVCTN2OpjtmoAn3ebL8uxHkQAEUl-ra2bGPUiiqPRfhqInmOXvBP6WG8QFLzXVShUTqCAOdKVZ3AgloWmVu1ivouYzjaVyzX297FgZi71zDSDimGc8s1xrX0pLtL9vcSf7x8dyYAmly_p3qb1p6aUnn-YBu66lmZKz6b-S8uc', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': -34.6683431, 'lng': 150.8549502}, 'viewport': {'northeast': {'lat': -34.6670395197085, 'lng': 150.8561457302915}, 'southwest': {'lat': -34.6697374802915, 'lng': 150.8534477697085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'name': 'The Sebel Kiama Harbourside', 'opening_hours': {'open_now': True}, 'photos': [{'height': 768, 'html_attributions': ['<a href="https://maps.google.com/maps

{'html_attributions': [], 'next_page_token': 'AcnaJv2ILHUt1t1DYD4HAZOyp1KPCKxBrJGao9ixg5sy1nTk4o9sWCKlzgczevi_t2MP0qQ0Myjtasl0N8uhLFYP7KNz48EiSPLNxaH4kkwqonBMa0oQ3Ebama8n0DEGoqXmBx_8Kv51K4ybbDlmOIu7rHoT51mZv2rZOnsUqSf9OG2lhKtxEKne6TsK-zkFswfFy62xbJEzrdFH-qAj2afbejAJKQ3ypw1pivK2ETwZnXt9vnDNq_-XtXM8nn-w_0gM7wS708KDdOKyqAKxNxGIkELiGJY_asu8JbrREPqj4AiAS4ceL9tmqVzvovHBpIcU3KIfH-0mOV_ppgZ62pShHd0MzLOe_8QZAr0MWVaQ5xDXkNQZ5wirbGulNZW8Ubo0Mgi_U40pw_gp8oHknmYAGySPvRq_M46qrvUUFZrVrk2v9vpiI6Q', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': -37.8147221, 'lng': 144.9417941}, 'viewport': {'northeast': {'lat': -37.8133801197085, 'lng': 144.9431253302915}, 'southwest': {'lat': -37.8160780802915, 'lng': 144.9404273697085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'name': 'AKOM AT DOCKLANDS', 'opening_hours': {'open_now': False}, 'photos': [{'height': 1367, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib

{'html_attributions': [], 'next_page_token': 'AcnaJv0PDUEMeAmuSBjtF-b9YOZ__CMnwa7l1XlPB9PNt113sPRMM0PX1LrYo7eCGQDI_DZ264Ug_5hscsUEsWwaNWx8s5q-2tVOesoCxVp1963q8qnGpjgh3LePstkV8gTWwLh-2XpSKYR_-dWjjG77rbApoz5qpS4Ywn6V-CJlmBpTQExdvV-RRCv1y6BLuWifRa-Nrl_sIb6cnJCGjIWygJ2N8LaaUgS3tiQpYRclRafyOkLovijo9sIvRWX-3EXjAo86EDLPbXEg3SaO0Zgh_jUzVfv0ycwNUcNLKxd1hjNt9fsR7j2Sm2paAYV6-8_l4FjfRFcDmdxVGBzvFCdaYc8Jv3qbbvhwVbz9ok6Kzhyni4g47NalK9GUUSWm3Xmrbtiq-1_ZYmoK7jyYRcXOfSaA7crhsM5rCAdlD_JTR91AG3kmniY', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': -32.488861, 'lng': 137.7631378}, 'viewport': {'northeast': {'lat': -32.4875278197085, 'lng': 137.7645001302915}, 'southwest': {'lat': -32.4902257802915, 'lng': 137.7618021697085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'name': 'Majestic Oasis Apartments', 'opening_hours': {'open_now': True}, 'photos': [{'height': 683, 'html_attributions': ['<a href="https://maps.google.com/maps/co

In [138]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [139]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)


# Add layer
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))